In [28]:
import pandas as pd
# Define a function to load data dynamically
def load_data(file_path):
    # Load data from each sheet into separate dataframes
    locations_df = pd.read_excel(file_path, sheet_name='locations', usecols="A").dropna()
    nurses_df = pd.read_excel(file_path, sheet_name='nurses', usecols="A:B").dropna()
    patients_df = pd.read_excel(file_path, sheet_name='patients', usecols="A:C").dropna()
    task_execution_time_df = pd.read_excel(file_path, sheet_name='task_execution_time', usecols="A:B").dropna()
    medication_adherence_df = pd.read_excel(file_path, sheet_name='medication_adherence', usecols="A:H").replace('Not Applicable', pd.NA).dropna(how='all', subset=['M', 'T', 'W', 'Th', 'F', 'S', 'Su'])
    physical_therapy_adherence_df = pd.read_excel(file_path, sheet_name='physical_therapy_adherence', usecols="A:H").replace('Not Applicable', pd.NA).dropna(how='all', subset=['M', 'T', 'W', 'Th', 'F', 'S', 'Su'])
    distance_matrix_df = pd.read_excel(file_path, sheet_name='distance_matrix', header=None).dropna(how='all')

    # Process data
    task_execution_time_df['Time'] = task_execution_time_df['Time'].str.extract('(\d+)').astype(int)
    nurses_df['skillset'] = nurses_df['skillset'].str.split(', ')
    patients_df['needs'] = patients_df['needs'].str.split(', ')

    for col in ['M', 'T', 'W', 'Th', 'F', 'S', 'Su']:
        medication_adherence_df[col] = pd.to_numeric(medication_adherence_df[col] * 100, errors='coerce')
        physical_therapy_adherence_df[col] = pd.to_numeric(physical_therapy_adherence_df[col] * 100, errors='coerce')

    for col in ['M', 'T', 'W', 'Th', 'F', 'S', 'Su']:
        medication_adherence_df[col] = (medication_adherence_df[col]).astype('Int64', errors='ignore')
        physical_therapy_adherence_df[col] = (physical_therapy_adherence_df[col]).astype('Int64', errors='ignore')
    
    
    return {
        "locations": locations_df,
        "nurses": nurses_df,
        "patients": patients_df,
        "task_execution_time": task_execution_time_df,
        "medication_adherence": medication_adherence_df,
        "physical_therapy_adherence": physical_therapy_adherence_df,
        "distance_matrix": distance_matrix_df
    }


# Load data dynamically
file_path = './nurse_schedule_data_small_VA (1).xlsx'
data = load_data(file_path)

# Extract individual dataframes from the data dictionary
locations_df = data["locations"]
nurses_df = data["nurses"]
patients_df = data["patients"]
task_execution_time_df = data["task_execution_time"]
medication_adherence_df = data["medication_adherence"]
physical_therapy_adherence_df = data["physical_therapy_adherence"]
distance_matrix_df = data["distance_matrix"]

# Display the first few rows of each dataframe to verify the dynamic data loading
for name, df in data.items():
    print(f"{name}:\n{df.head()}\n")

days = ['M', 'T', 'W', 'Th', 'F', 'S', 'Su']


locations:
     Unnamed: 0
0       Roanoke
1  Williamsburg
2       Norfolk
3    Chesapeake

nurses:
        id                                           skillset
0  Nurse_1  [medication, personal hygiene assistance, phys...
1  Nurse_2  [medication, administering injections, physica...
2  Nurse_3  [physical therapy, medication, administering i...
3  Nurse_4  [wound care, personal hygiene assistance, draw...
4  Nurse_5  [medication, personal hygiene assistance, draw...

patients:
          id                                   needs      location
0  Patient_1  [administering injections, medication]       Norfolk
1  Patient_2                            [wound care]       Roanoke
2  Patient_3                            [medication]  Williamsburg
3  Patient_4              [administering injections]  Williamsburg
4  Patient_5                            [wound care]       Norfolk

task_execution_time:
                       Task  Time
0                medication    37
1             drawing blo

In [29]:
from gurobipy import Model, GRB, quicksum
import numpy as np

model = Model("nurse_scheduling")

# Get the number of nurses, patients, tasks, and days
day_dict = {'M': 0, 'T': 1, 'W': 2, 'Th': 3, 'F': 4, 'S': 5, 'Su': 6}
num_nurses = len(nurses_df)
num_patients = len(patients_df)
num_tasks = len(task_execution_time_df)
num_days = 7  # Days in a week
task_list = task_execution_time_df['Task'].tolist()
#nurse_skills = nurses_df.iloc[i]['skillset']
# Convert adherence dataframes to dictionaries for easier access
medication_adherence_dict = medication_adherence_df.set_index('id').T.to_dict('dict')
physical_therapy_adherence_dict = physical_therapy_adherence_df.set_index('id').T.to_dict('dict')


# Define Decision Variables
#x[i, j, k]: A binary variable that determines whether nurse i is assigned to patient j on day k.
x = model.addVars(num_nurses, num_patients, num_days, vtype=GRB.BINARY, name="x")
#w[i, k]: A binary variable that indicates whether nurse i is working on day k.
w = model.addVars(num_nurses, num_days, vtype=GRB.BINARY, name="w")
# p[i, m, k]: A binary variable that indicates whether nurse i is at location m on day k.
p = model.addVars(num_nurses, len(locations_df), num_days, vtype=GRB.BINARY, name="p")
#t[i, j, n, k]: A binary variable that indicates whether nurse i is performing task n for patient j on day k.
t = model.addVars(num_nurses, num_patients, num_tasks, num_days, vtype=GRB.BINARY, name="t")

# Define a function to get adherence value
def get_adherence(patient_id, task, day):
    if task == 'medication':
        return medication_adherence_dict[patient_id][day] if patient_id in medication_adherence_dict and day in medication_adherence_dict[patient_id] else 0
    elif task == 'physical therapy':
        return physical_therapy_adherence_dict[patient_id][day] if patient_id in physical_therapy_adherence_dict and day in physical_therapy_adherence_dict[patient_id] else 0
    else:
        return 0

# Define the objective function
model.setObjective(
    quicksum(
        t[i, j, task_list.index(task), d] + get_adherence(patients_df.iloc[j]['id'], task, list(day_dict.keys())[d]) * (1 - t[i, j, task_list.index(task), d])
        for i in range(num_nurses)
        for j in range(num_patients)
        for task in task_list
        for d in range(num_days)
    ),
    GRB.MAXIMIZE,
)

# Convert task execution time dataframe to dictionary for easier access
task_time_dict = task_execution_time_df.set_index('Task').to_dict()['Time']


# Adding the constraint that each nurse can work up to 40 hours in a week
for i in range(num_nurses):
    model.addConstr(
        quicksum(
            t[i, j, k, d] * task_time_dict[task_list[k]]
            for j in range(num_patients)
            for k in range(num_tasks)
            for d in range(num_days)
        ) <= 40 * 60,  # 40 hours converted to minutes
        name=f"weekly_work_hours_nurse_{i}"
    )


# Adding constraint to ensure all mandatory tasks are covered
for j, patient_row in patients_df.iterrows():
    for task in patient_row['needs']:
        task_index = task_execution_time_df[task_execution_time_df['Task'] == task].index[0]  # Get the index of the task in the task_execution_time_df
        model.addConstr(quicksum(t[i, j, task_index, k] for i in range(num_nurses) for k in range(num_days)) >= 1, name=f"mandatory_task_coverage_patient_{j}_task_{task_index}")

for i in range(num_nurses):
    for k in range(num_days):
        # This constraint ensures that if a nurse is not working on a particular day (w[i, k] = 0), 
        # then the nurse cannot be assigned any patients or tasks on that day 
        # (all corresponding x[i, j, k] and t[i, j, l, k] must be 0).
        model.addConstr(quicksum(x[i, j, k] for j in range(num_patients)) + quicksum(t[i, j, l, k] for j in range(num_patients) for l in range(num_tasks)) <= w[i, k] * (num_patients * num_tasks), name=f"working_day_link_{i}_{k}")

        # This constraint ensures that if a nurse is assigned to at least one patient or task on a particular day 
        # (any of the corresponding x[i, j, k] or t[i, j, l, k] is 1), 
        # then the working day variable for the nurse on that day must be activated (w[i, k] must be 1).
        model.addConstr(w[i, k] <= quicksum(x[i, j, k] for j in range(num_patients)) + quicksum(t[i, j, l, k] for j in range(num_patients) for l in range(num_tasks)), name=f"working_day_link_reverse_{i}_{k}")


#the sum of nurses day's working is <= 4
for i in range(num_nurses):
    model.addConstr(quicksum(w[i, k] for k in range(num_days)) <= 4, name=f"max_working_days_nurse_{i}")


# Add constraints to only assign necessary tasks to patients and ensure the task is in nurse's skillset
for i in range(num_nurses):
    nurse_skills = nurses_df.iloc[i]['skillset']
    for j in range(num_patients):
        for l in range(num_tasks):
            if task_list[l] not in patients_df.at[j, 'needs'] or task_list[l] not in nurse_skills:
                model.addConstr(quicksum(t[i, j, l, k] for k in range(num_days)) == 0)

                
# Get a dictionary with patient IDs as keys and lists of required tasks as values
patient_tasks_dict = patients_df.set_index('id')['needs'].to_dict()


# Constraint ensuring each nurse works in only one location in a given day
for i in range(num_nurses):
    for d in range(num_days):
        model.addConstr(
            quicksum(p[i, m, d] for m in range(len(locations_df))) <= 1,
            name=f"Nurse_{i}_Day_{d}_One_Location"
        )


# Constraints to link the p variable with the t variable
for i in range(num_nurses):
    for j in range(num_patients):
        m = locations_df.index[locations_df['Unnamed: 0'] == patients_df.iloc[j]['location']].tolist()[0]  # Getting the index of patient's location
        for n in range(num_tasks):
            for d in range(num_days):
                model.addConstr(
                    p[i, m, d] >= t[i, j, n, d],
                    name=f"Link_Location_Task_Nurse_{i}_Patient_{j}_Task_{n}_Day_{d}"
                )


# This constraint ensures that each necessary task for each patient is completed each day.
# It sums over the t variable for each task and requires that the sum be equal to the 
# number of patients that need that task performed on each day of the week.
for j, patient_id in enumerate(patients_df['id']):
    for l, task in enumerate(task_execution_time_df['Task']):
        if task not in ['medication_adherence', 'physical_therapy_adherence']:
            model.addConstr(
                quicksum(t[i, j, l, k] for i in range(num_nurses) for k in range(num_days)) == 
                (task in patient_tasks_dict[patient_id]) * num_days, 
                name=f"task_completion_{patient_id}_{task}"
            )


#only 1 nurse is doing a certain task for a certain patient on a certain day
for j in range(num_patients):
    for k in range(num_tasks):
        for d in range(num_days):
            model.addConstr(quicksum(t[i, j, k, d] for i in range(num_nurses)) <= 1, 
                            name=f"single_nurse_per_task_per_day_patient_{j}_task_{k}_day_{d}")


# restricts nurses to only work in same location for consecutive days
for i in range(num_nurses):
    for m in range(len(locations_df)):
        for d in range(num_days - 1):  # Exclude the last day
            model.addConstr(
                p[i, m, d] + w[i, d+1] - p[i, m, d+1] <= 1,
                name=f"Consecutive_Location_Nurse_{i}_Location_{m}_Day_{d}"
            )
            

            
model.optimize()




if model.status == GRB.OPTIMAL:
    print('Objective Value: ', model.objVal)
    for i in range(num_nurses):
        # Get the skills of the nurse
        nurse_skills = nurses_df.iloc[i]['skillset']
        
        # Calculate total hours worked in the week for the nurse
        total_minutes_worked = sum(
            t[i, j, k, d].x * task_time_dict[task_list[k]]
            for j in range(num_patients)
            for k in range(num_tasks)
            for d in range(num_days)
        )
        total_hours_worked = total_minutes_worked / 60  # Convert minutes to hours

        print(f'Nurse_{i+1} \n Skills: {nurse_skills} \n Total Hours Worked: {total_hours_worked:.2f}\n')
        for k in range(num_days):
            working = False
            tasks_for_the_day = []
            daily_minutes_worked = 0  # Initialize daily_minutes_worked variable
            for j in range(num_patients):
                for l in range(num_tasks):
                    if t[i, j, l, k].x > 0.5:  # If the task is assigned
                        location = patients_df.iloc[j]['location']
                        task_name = task_list[l]
                        tasks_for_the_day.append((j, task_name, location))
                        daily_minutes_worked += task_time_dict[task_name]  # Add time for the task to daily total
                        working = True
            daily_hours_worked = daily_minutes_worked / 60  # Convert daily_minutes_worked to hours
            if working:
                print(f'  Day {list(day_dict.keys())[k]}: Working (Total Hours Worked: {daily_hours_worked:.2f})')
                for task in tasks_for_the_day:
                    print(f'    - Helping Patient_{task[0]+1} with {task[1]} in {task[2]}')
                print('\n')
            else:
                print(f'  Day {list(day_dict.keys())[k]}: Off\n')
else:
    print('No optimal solution found. Status code:', model.status)

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 15453 rows, 15225 columns and 105975 nonzeros
Model fingerprint: 0x745ba8fa
Variable types: 0 continuous, 15225 integer (15225 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+01]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective 61292.000000
Presolve removed 11949 rows and 12000 columns
Presolve time: 0.04s
Presolved: 3504 rows, 3225 columns, 16806 nonzeros
Variable types: 0 continuous, 3225 integer (3225 binary)

Root relaxation: cutoff, 792 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0      61292.0000 61292.0000  0.00%     

In [30]:
# 1. Nurse Working Hours Check
for i in range(num_nurses):
    total_minutes_worked = sum(
        t[i, j, k, d].x * task_time_dict[task_list[k]]
        for j in range(num_patients)
        for k in range(num_tasks)
        for d in range(num_days)
    )
    assert total_minutes_worked <= 40 * 60, f"Nurse {i} works more than 40 hours in a week."


In [31]:
# 2. Task Completion Check
for j in range(num_patients):
    patient_tasks = patients_df.iloc[j]['needs']
    for task in patient_tasks:
        task_index = task_list.index(task)
        assigned_times = sum(t[i, j, task_index, d].x for i in range(num_nurses) for d in range(num_days))
        assert assigned_times == num_days, f"Patient {j}'s task {task} not completed daily."


In [32]:
# 3. Single Location Check
for i in range(num_nurses):
    for d in range(num_days):
        locations_worked = sum(p[i, m, d].x for m in range(len(locations_df)))
        assert locations_worked <= 1, f"Nurse {i} works in more than one location on day {d}."


In [33]:
# 4. Skill Matching Check
for i in range(num_nurses):
    nurse_skills = nurses_df.iloc[i]['skillset']
    for j in range(num_patients):
        for k in range(num_tasks):
            if t[i, j, k, d].x > 0.5 and task_list[k] not in nurse_skills:
                raise AssertionError(f"Nurse {i} assigned task {task_list[k]} which is not in their skillset.")


In [34]:
# 5. Nurse's Work Days Check
for i in range(num_nurses):
    days_worked = sum(w[i, d].x for d in range(num_days))
    assert days_worked <= 4, f"Nurse {i} works more than 4 days in a week."


In [35]:
# 6. Only One Nurse Per Task Per Day Check
for j in range(num_patients):
    for k in range(num_tasks):
        for d in range(num_days):
            nurses_assigned = sum(t[i, j, k, d].x for i in range(num_nurses))
            assert nurses_assigned <= 1, f"More than one nurse assigned to Patient {j} for Task {k} on Day {d}."
